# An Initial Exploration of LIV + UV KASH cells image analysis

## Data Loading and Preprocessing

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import scipy.stats as stats
from scipy.stats import mannwhitneyu
from itertools import combinations
from statannotations.Annotator import Annotator

In [ ]:
file_path = r"D:\BioLab\Current_experiments\Anamaria_3d_experiment\02.08.2023_analysis_data\cell_stat.csv"
initial_df = pd.read_csv(file_path)
initial_df = initial_df.dropna(how='all')
initial_df

In [ ]:
def extract_info(image_name):
    date_pattern = re.compile(r'\d+-\d+-\d+')
    date = date_pattern.search(image_name).group(0)

    doxy = "+doxy" if "+doxy" in image_name else "-doxy"

    liv = "+LIV" if "+LIV" in image_name else "-LIV"

    treatment = "+Cisp" if "+Cisp" in image_name else "Control"

    time_pattern = re.compile(r'0hr|24hr|48hr')
    time = time_pattern.search(image_name).group(0)

    return pd.Series([date, doxy, liv, treatment, time], index=['Date', 'Doxy', 'LIV', 'Treatment', 'Time'])



In [ ]:
# Extract the information into temporary DataFrame
temp_df = initial_df['Image_name'].apply(extract_info)

# Find the index of the "Image_name" column
idx = initial_df.columns.get_loc('Image_name') + 1

# Insert the new columns
for col in reversed(temp_df.columns):
    initial_df.insert(loc=idx, column=col, value=temp_df[col])


In [ ]:
initial_df

In [ ]:
initial_df.columns

In [ ]:
filtered_df = initial_df[['Image_name', 'Img_num','Cell_num','Doxy', 'LIV', 'Treatment', 'Time', 'Nucleus_volume, cubic_micrometre',
       'Nucleus_length, micrometre', 'Nucleus_width, micrometre',
       'Nucleus_high, micrometre', 'AF594-T2 av_signal_in_nuc_area_3D', 'AF594-T2 has ring',
       'AF594-T2 ring intensity coef', 'AF488-T3 av_signal_in_nuc_area_3D']]

In [ ]:
correlation_matrix = filtered_df.corr()
plt.figure(figsize=(12, 9)) # Set the size to 15 inches wide and 10 inches tall
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
filtered_df['Group'] = filtered_df['Doxy'] + ', ' + filtered_df['LIV'] + ', ' + filtered_df['Treatment'] + ', ' + filtered_df['Time']
plt.figure(figsize=(15, 10))
sns.boxplot(x='Group', y='AF488-T3 av_signal_in_nuc_area_3D', data=filtered_df)
plt.title('Distribution of AF488-T3 av_signal_in_nuc_area_3D across groups \nWithout ring/no-ring correction')
plt.xticks(rotation=45) # Rotate x-axis labels for better visibility
plt.savefig('figures/Distribution of AF488-T3 av_signal_in_nuc_area_3D across groups.png', dpi=300, bbox_inches='tight')
plt.show()

## Signal background

Acquire control cells and measure the average signal at 0h and 48 hours for all cells. This signal will then be subtracted from the average signal of each cell subsequently.

In [ ]:
cnt_signal = filtered_df[filtered_df['Treatment'] == 'Control']['AF488-T3 av_signal_in_nuc_area_3D'].mean()
cnt_signal = 0

In [ ]:
filtered_df['AF488-T3 av_signal_in_nuc_area_3D_removed_background'] = filtered_df['AF488-T3 av_signal_in_nuc_area_3D'] - cnt_signal
a = filtered_df[filtered_df['Treatment'] == '+Cisp']
a[a['AF488-T3 av_signal_in_nuc_area_3D_removed_background'] < 0]

## Total repair signal within nucleus area

New column titled "AF488-T3 Volume_corrected_signal_in_nuc_area_3D." This will display the repair signal within each nucleus in its entirety. The value is derived from the background-corrected average signal.

In [ ]:
resolution = 0.085 * 0.058 * 0.230 #x.resolution * y.resolution * z.resolution
filtered_df['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'] = filtered_df['AF488-T3 av_signal_in_nuc_area_3D_removed_background'] * filtered_df['Nucleus_volume, cubic_micrometre']

In [ ]:
import csv
RING_CUTT_OFF_COEF = 1
filtered_df['AF594-T2 has ring'] = filtered_df['AF594-T2 ring intensity coef'] >= RING_CUTT_OFF_COEF

with pd.ExcelWriter('raw_data.xlsx') as writer:
    filtered_df.to_excel(writer, index=False)


filtered_df.to_csv('raw_data.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))

# Extract the order in which seaborn plots the groups
order = sorted(filtered_df['Group'].unique())

# Boxplot with fixed order
sns.boxplot(x='Group', y='AF488-T3 Volume_corrected_signal_in_nuc_area_3D', data=filtered_df, order=order)

# Calculate the means and SDs for each group
group_means = filtered_df.groupby('Group')['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].mean().reindex(order)
group_stds = filtered_df.groupby('Group')['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].std().reindex(order)

# Define a consistent height for all the text labels
label_height = 2000000  # Some margin above the maximum value

# Display the means and SDs at the consistent height
for i, (group, mean) in enumerate(group_means.iteritems()):
    plt.text(i, label_height, f"Mean: {mean:.0f}", ha='center', color='red', fontsize=10)
    plt.text(i, label_height - 200000, f"SD: ±{group_stds[group]:.0f}", ha='center', color='blue', fontsize=10)

plt.title('Distribution of AF488-T3 Volume_corrected_signal_in_nuc_area_3D \nNo background correction \nWithout ring/no-ring correction')
plt.xticks(rotation=45) # Rotate x-axis labels for better visibility
plt.savefig('figures/Distribution of AF488-T3 Volume_corrected_signal_in_nuc_area_3D.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define the groups
groups_true = [
    '+doxy, +LIV, +Cisp, 0hr',
    '+doxy, +LIV, +Cisp, 48hr',
    '+doxy, -LIV, +Cisp, 0hr',
    '+doxy, -LIV, +Cisp, 48hr',
    '+doxy, -LIV, Control, 0hr',
    '+doxy, -LIV, Control, 48hr'
]

groups_false = [
    '-doxy, +LIV, +Cisp, 0hr',
    '-doxy, +LIV, +Cisp, 48hr',
    '-doxy, -LIV, +Cisp, 0hr',
    '-doxy, -LIV, +Cisp, 48hr'
]

# Filter the DataFrame based on conditions
subset_true = filtered_df[(filtered_df['Group'].isin(groups_true)) & (filtered_df['AF594-T2 has ring'] == True)]
subset_false = filtered_df[(filtered_df['Group'].isin(groups_false)) & (filtered_df['AF594-T2 has ring'] == False)]

# Concatenate both dataframes for a unified plot
combined_df = pd.concat([subset_true, subset_false], ignore_index=True)

# Define colors for each group
colors = sns.color_palette("husl", len(groups_true + groups_false))

# Plot combined boxplots with individual colors
plt.figure(figsize=(20, 10))
sns.boxplot(x='Group', y='AF488-T3 Volume_corrected_signal_in_nuc_area_3D', data=combined_df, palette=colors)

# Labeling
combined_means = combined_df.groupby('Group')['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].mean().reindex(groups_true + groups_false)
combined_stds = combined_df.groupby('Group')['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].std().reindex(groups_true + groups_false)

label_height =  2000000

for i, (group, mean) in enumerate(combined_means.iteritems()):
    plt.text(i, label_height, f"{mean:.0f}", ha='center', color='red', fontsize=15)
    plt.text(i, label_height - 200000, f"±{combined_stds[group]:.0f}", ha='center', color='blue', fontsize=15)

plt.title('Combined Distribution of AF488-T3 Volume_corrected_signal_in_nuc_area_3D \nAdjusted to a ring presence: \n+doxy with a ring, -doxy without a ring \n\n')
plt.xticks(rotation=45)
plt.savefig('figures/Distribution of AF488-T3 Volume_corrected_signal_in_nuc_area_3D_ring_no-ring.png', dpi=300, bbox_inches='tight')

plt.show()


## -Doxy + Cisp +LIV/-LIV 0hr/48hr

In [ ]:
minus_doxy_plus_cisp_df = filtered_df[(filtered_df['Doxy'] == '-doxy') & (filtered_df['Treatment'] == '+Cisp')]
minus_doxy_plus_cisp_df

In [ ]:
# Custom colors
blue = (0.298, 0.447, 0.690)  # a desaturated blue color
orange = (1.0, 0.498, 0.055)  # a desaturated orange color
pal = [blue, orange]

sns.set(font_scale=2, rc={'axes.facecolor':(0,0,0,0), 'figure.facecolor':(0,0,0,0)}, style="ticks")

significanceComparisons = [(('+LIV', '0hr'), ('+LIV', '48hr')),
                           (('-LIV', '0hr'), ('-LIV','48hr'))]

fig_args = {'x': 'LIV',
            'y': 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D',
            'hue':'Time',
            'data': minus_doxy_plus_cisp_df,
            'dodge': True,
            'palette': pal}

configuration = {'test':'Mann-Whitney',
                 'comparisons_correction':None,
                 'text_format':'star'}

fig, ax = plt.subplots(1,1, figsize=(15,10))

sns.barplot(ax=ax, **fig_args)
annotator = Annotator(ax=ax, pairs=significanceComparisons,
                      **fig_args)
annotator.configure(**configuration).apply_test().annotate()


# Move legend and remove its background
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
legend = ax.legend_
legend.get_frame().set_alpha(0)

In [ ]:
filtered_df

## -Doxy +LIV +Cisp


In [ ]:
# Filter DataFrame to only include '- Doxy +Crisp +LIV' group without rings
minus_doxy_plus_cisp_plus_liv_no_ring_df = filtered_df[(filtered_df['Doxy'] == '-doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '+LIV') & (filtered_df['AF594-T2 ring intensity coef'] < RING_CUTT_OFF_COEF)].copy()

minus_doxy_plus_cisp_plus_liv_df = filtered_df[(filtered_df['Doxy'] == '-doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '+LIV') ].copy()

# Normalize 'AF488-T3 av_signal_in_nuc_area_3D' by average of '0hr' group
average_0hr = minus_doxy_plus_cisp_plus_liv_no_ring_df[minus_doxy_plus_cisp_plus_liv_no_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].mean()
minus_doxy_plus_cisp_plus_liv_no_ring_df.loc[:, 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D'] /= average_0hr
print(f"Total -Doxy +LIV +Cisp: {len(minus_doxy_plus_cisp_plus_liv_df)}")
print(f"No ring -Doxy +LIV +Cisp: {len(minus_doxy_plus_cisp_plus_liv_no_ring_df)} = "
      f"0hr: {len(minus_doxy_plus_cisp_plus_liv_no_ring_df[minus_doxy_plus_cisp_plus_liv_no_ring_df['Time'] == '0hr'])} + "
      f"48hr: {len(minus_doxy_plus_cisp_plus_liv_no_ring_df[minus_doxy_plus_cisp_plus_liv_no_ring_df['Time'] == '48hr'])}")

In [ ]:
# Extract data for '0hr' and '48hr' groups
group_0hr = minus_doxy_plus_cisp_plus_liv_no_ring_df[minus_doxy_plus_cisp_plus_liv_no_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']
group_48hr = minus_doxy_plus_cisp_plus_liv_no_ring_df[minus_doxy_plus_cisp_plus_liv_no_ring_df['Time'] == '48hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']

group_48hr_minus_doxy_plus_liv_plus_cisp = minus_doxy_plus_cisp_plus_liv_no_ring_df[minus_doxy_plus_cisp_plus_liv_no_ring_df['Time'] == '48hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']

# Perform the Mann-Whitney U test
stat, p = mannwhitneyu(group_0hr, group_48hr)

# Print the p-value
print('p-value:', p)


In [ ]:
# Adjust significance comparisons for '+LIV' group only
significanceComparisons = [('0hr','48hr')]

fig_args = {'x': 'Time',
            'y': 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D',
            'data': minus_doxy_plus_cisp_plus_liv_no_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))

sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('-Doxy +LIV +Cisp \nNo ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('AF488-T3 signal corrected to nuc Volume', fontsize=20)

plt.savefig('figures/-Doxy +LIV +Cisp.png', dpi=300, bbox_inches='tight')

In [ ]:
minus_doxy_plus_cisp_plus_liv_no_ring_df.columns

In [ ]:
# Adjust significance comparisons for '+LIV' group only
significanceComparisons = [('0hr','48hr')]

fig_args = {'x': 'Time',
            'y': 'Nucleus_volume, cubic_micrometre',
            'data': minus_doxy_plus_cisp_plus_liv_no_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))

sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('-Doxy +LIV +Cisp \nNo ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('Nucleus volume, µm3', fontsize=20)

plt.savefig('figures/-Doxy +LIV +Cisp Nucleus_volume.png', dpi=300, bbox_inches='tight')

## +Doxy +LIV +Cisp

In [ ]:
# Filter DataFrame to only include '- Doxy +Crisp +LIV' group without rings
plus_doxy_plus_cisp_plus_liv_with_ring_df = filtered_df[(filtered_df['Doxy'] == '+doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '+LIV') & (filtered_df['AF594-T2 ring intensity coef'] >= RING_CUTT_OFF_COEF)].copy()

plus_doxy_plus_cisp_plus_liv_df = filtered_df[(filtered_df['Doxy'] == '+doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '+LIV') ].copy()

# Normalize 'AF488-T3 av_signal_in_nuc_area_3D' by average of '0hr' group
average_0hr = plus_doxy_plus_cisp_plus_liv_with_ring_df[plus_doxy_plus_cisp_plus_liv_with_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].mean()
plus_doxy_plus_cisp_plus_liv_with_ring_df.loc[:, 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D'] /= average_0hr
print(f"Total +Doxy +LIV +Cisp: {len(plus_doxy_plus_cisp_plus_liv_df)}")

print(f"With ring +Doxy +LIV +Cisp: {len(plus_doxy_plus_cisp_plus_liv_with_ring_df)} = "
      f"0hr: {len(plus_doxy_plus_cisp_plus_liv_with_ring_df[plus_doxy_plus_cisp_plus_liv_with_ring_df['Time'] == '0hr'])} + "
      f"48hr: {len(plus_doxy_plus_cisp_plus_liv_with_ring_df[plus_doxy_plus_cisp_plus_liv_with_ring_df['Time'] == '48hr'])}")

In [ ]:
# Extract data for '0hr' and '48hr' groups
group_0hr = plus_doxy_plus_cisp_plus_liv_with_ring_df[plus_doxy_plus_cisp_plus_liv_with_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']
group_48hr = plus_doxy_plus_cisp_plus_liv_with_ring_df[plus_doxy_plus_cisp_plus_liv_with_ring_df['Time'] == '48hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']

# Perform the Mann-Whitney U test
stat, p = mannwhitneyu(group_0hr, group_48hr)

# Print the p-value
print('p-value:', p)

In [ ]:
fig_args = {'x': 'Time',
            'y': 'Nucleus_volume, cubic_micrometre',
            'data': plus_doxy_plus_cisp_plus_liv_with_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))

sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('+Doxy +LIV +Cisp \nWith ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('Nucleus_volume, µm3', fontsize=20)

plt.savefig('figures/+Doxy +LIV +Cisp Nucleus_volume.png', dpi=300, bbox_inches='tight')

In [ ]:
fig_args = {'x': 'Time',
            'y': 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D',
            'data': plus_doxy_plus_cisp_plus_liv_with_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))

sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('+Doxy +LIV +Cisp \nWith ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('AF488-T3 signal corrected to nuc Volume', fontsize=20)

plt.savefig('figures/+Doxy +LIV +Cisp.png', dpi=300, bbox_inches='tight')

## -Doxy -LIV +Cisp

In [ ]:
# Filter DataFrame to only include '- Doxy +Crisp +LIV' group without rings
minus_doxy_plus_cisp_minus_liv_no_ring_df = filtered_df[(filtered_df['Doxy'] == '-doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '-LIV') & (filtered_df['AF594-T2 ring intensity coef'] < RING_CUTT_OFF_COEF)].copy()

minus_doxy_plus_cisp_minus_liv_df = filtered_df[(filtered_df['Doxy'] == '-doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '-LIV') ].copy()

# Normalize 'AF488-T3 av_signal_in_nuc_area_3D' by average of '0hr' group
average_0hr = minus_doxy_plus_cisp_minus_liv_no_ring_df[minus_doxy_plus_cisp_minus_liv_no_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].mean()
minus_doxy_plus_cisp_minus_liv_no_ring_df.loc[:, 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D'] /= average_0hr
print(f"Total -Doxy -LIV +Cisp: {len(minus_doxy_plus_cisp_minus_liv_df)}")
print(f"No ring -Doxy -LIV +Cisp: {len(minus_doxy_plus_cisp_minus_liv_no_ring_df)} = "
      f"0hr: {len(minus_doxy_plus_cisp_minus_liv_no_ring_df[minus_doxy_plus_cisp_minus_liv_no_ring_df['Time'] == '0hr'])} + "
      f"48hr: {len(minus_doxy_plus_cisp_minus_liv_no_ring_df[minus_doxy_plus_cisp_minus_liv_no_ring_df['Time'] == '48hr'])}")

In [ ]:
# Extract data for '0hr' and '48hr' groups
group_0hr = minus_doxy_plus_cisp_minus_liv_no_ring_df[minus_doxy_plus_cisp_minus_liv_no_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']
group_48hr = minus_doxy_plus_cisp_minus_liv_no_ring_df[minus_doxy_plus_cisp_minus_liv_no_ring_df['Time'] == '48hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']

# Perform the Mann-Whitney U test
stat, p = mannwhitneyu(group_0hr, group_48hr)

print(f"P-value for volume: {p}")

In [ ]:
fig_args = {'x': 'Time',
            'y': 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D',
            'data': minus_doxy_plus_cisp_minus_liv_no_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))

sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('-Doxy -LIV +Cisp \nNo ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('AF488-T3 signal corrected to nuc Volume', fontsize=20)

plt.savefig('figures/-Doxy -LIV +Cisp.png', dpi=300, bbox_inches='tight')

In [ ]:
fig_args = {'x': 'Time',
            'y': 'Nucleus_volume, cubic_micrometre',
            'data': minus_doxy_plus_cisp_minus_liv_no_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))

sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('-Doxy -LIV +Cisp \nNo ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('Nucleus_volume, µm3', fontsize=20)

plt.savefig('figures/-Doxy -LIV +Cisp Nucleus_volume, cubic_micrometre.png', dpi=300, bbox_inches='tight')

## +Doxy -LIV +Cisp

In [ ]:
# Filter DataFrame to only include '+Doxy +Crisp -LIV' group with rings
plus_doxy_plus_cisp_minus_liv_with_ring_df = filtered_df[(filtered_df['Doxy'] == '+doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '-LIV') & (filtered_df['AF594-T2 ring intensity coef'] >= RING_CUTT_OFF_COEF)].copy()


plus_doxy_plus_cisp_minus_liv_df = filtered_df[(filtered_df['Doxy'] == '+doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['LIV'] == '-LIV') ].copy()

# Normalize 'AF488-T3 av_signal_in_nuc_area_3D' by average of '0hr' group
average_0hr = plus_doxy_plus_cisp_minus_liv_with_ring_df[plus_doxy_plus_cisp_minus_liv_with_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].mean()

plus_doxy_plus_cisp_minus_liv_with_ring_df.loc[:, 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D'] /= average_0hr


print(f"Total +Doxy -LIV +Cisp: {len(plus_doxy_plus_cisp_minus_liv_df)}")
print(f"With ring +Doxy -LIV +Cisp: {len(plus_doxy_plus_cisp_minus_liv_with_ring_df)} = "
      f"0hr: {len(plus_doxy_plus_cisp_minus_liv_with_ring_df[plus_doxy_plus_cisp_minus_liv_with_ring_df['Time'] == '0hr'])} + "
      f"48hr: {len(plus_doxy_plus_cisp_minus_liv_with_ring_df[plus_doxy_plus_cisp_minus_liv_with_ring_df['Time'] == '48hr'])}")


In [ ]:
plus_doxy_plus_cisp_minus_liv_with_ring_df

In [ ]:
# Extract data for '0hr' and '48hr' groups
group_0hr = plus_doxy_plus_cisp_minus_liv_with_ring_df[plus_doxy_plus_cisp_minus_liv_with_ring_df['Time'] == '0hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']
group_48hr = plus_doxy_plus_cisp_minus_liv_with_ring_df[plus_doxy_plus_cisp_minus_liv_with_ring_df['Time'] == '48hr']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']

# Perform the Mann-Whitney U test
stat, p = mannwhitneyu(group_0hr, group_48hr)

# Print the p-value
print('p-value:', p)

In [ ]:
fig_args = {'x': 'Time',
            'y': 'Nucleus_volume, cubic_micrometre',
            'data': plus_doxy_plus_cisp_minus_liv_with_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))
# Adjust significance comparisons for '+LIV' group only
significanceComparisons = [('0hr','48hr')]
sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('+Doxy -LIV +Cisp\nWith ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('Nucleus_volume, µm3', fontsize=20)

plt.savefig('figures/+Doxy -LIV +Cisp Nucleus_volume.png', dpi=300, bbox_inches='tight')

In [ ]:
fig_args = {'x': 'Time',
            'y': 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D',
            'data': plus_doxy_plus_cisp_minus_liv_with_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))
# Adjust significance comparisons for '+LIV' group only
significanceComparisons = [('0hr','48hr')]
sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('+Doxy -LIV +Cisp\nWith ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('AF488-T3 signal corrected to nuc Volume', fontsize=20)

plt.savefig('figures/+Doxy -LIV +Cisp.png', dpi=300, bbox_inches='tight')

## -Doxy -LIV +Cisp 48hr VS. -Doxy +LIV +Cisp 48hr

In [ ]:
# Filter DataFrame to only include '+Doxy +Crisp -LIV' group with rings
minus_doxy_plus_cisp_48hr_no_ring_df = filtered_df[(filtered_df['Doxy'] == '-doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['Time'] == '48hr') & (filtered_df['AF594-T2 ring intensity coef'] < RING_CUTT_OFF_COEF)].copy()
minus_doxy_plus_cisp_48hr = filtered_df[(filtered_df['Doxy'] == '-doxy') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['Time'] == '48hr')].copy()

# Normalize 'AF488-T3 av_signal_in_nuc_area_3D' by average of '-LIV' group
average_plus_liv = minus_doxy_plus_cisp_48hr_no_ring_df[minus_doxy_plus_cisp_48hr_no_ring_df['LIV'] == '+LIV']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D'].mean()

minus_doxy_plus_cisp_48hr_no_ring_df.loc[:, 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D'] /= average_plus_liv


print(f"Total -Doxy +Cisp 48hr: {len(minus_doxy_plus_cisp_48hr)}")
print(f"No ring -Doxy -LIV +Cisp: {len(minus_doxy_plus_cisp_48hr_no_ring_df)} = "
      f"-LIV: {len(minus_doxy_plus_cisp_48hr_no_ring_df[minus_doxy_plus_cisp_48hr_no_ring_df['LIV'] == '-LIV'])} + "
      f"+LIV: {len(minus_doxy_plus_cisp_48hr_no_ring_df[minus_doxy_plus_cisp_48hr_no_ring_df['LIV'] == '+LIV'])}")

In [ ]:
plus_doxy_plus_cisp_minus_liv_with_ring_df

In [ ]:
# Extract data for '-LIV' and '+LIV' groups
group_minus_liv = minus_doxy_plus_cisp_48hr_no_ring_df[minus_doxy_plus_cisp_48hr_no_ring_df['LIV'] == '-LIV']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']
group_plus_liv = minus_doxy_plus_cisp_48hr_no_ring_df[minus_doxy_plus_cisp_48hr_no_ring_df['LIV'] == '+LIV']['AF488-T3 Volume_corrected_signal_in_nuc_area_3D']

# Perform the Mann-Whitney U test
stat, p = mannwhitneyu(group_minus_liv, group_plus_liv)

# Print the p-value
print('p-value:', p)

In [ ]:
fig_args = {'x': 'LIV',
            'y': 'AF488-T3 Volume_corrected_signal_in_nuc_area_3D',
            'data': minus_doxy_plus_cisp_48hr_no_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))
# Adjust significance comparisons for '+LIV' group only
significanceComparisons = [('-LIV','+LIV')]
sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('-Doxy +Cisp 48hr\nNo ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('AF488-T3 signal corrected to nuc Volume', fontsize=20)

plt.savefig('figures/-Doxy +Cisp 48hr.png', dpi=300, bbox_inches='tight')


In [ ]:
minus_doxy_plus_cisp_48hr_no_ring_df.columns

In [ ]:
fig_args = {'x': 'LIV',
            'y': 'Nucleus_volume, cubic_micrometre',
            'data': minus_doxy_plus_cisp_48hr_no_ring_df,
            'dodge': True,
            'palette': pal}

fig, ax = plt.subplots(1,1, figsize=(10,10))
# Adjust significance comparisons for '+LIV' group only
significanceComparisons = [('-LIV','+LIV')]
sns.barplot(ax=ax, **fig_args)

# Annotate the bars with their actual mean values
for p in ax.patches:
    ax.annotate(f"{p.get_height():.2f}",
                (p.get_x(), p.get_height()),
                ha='left', va='center', fontsize=30, color='black',
                xytext=(5, 20),  # Add a slight horizontal offset to prevent overlap with the bar
                textcoords='offset points')

# You might need to import the StatAnnotator
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# To modify title size use
ax.set_title('-Doxy +Cisp 48hr\nNo ring\n', fontsize=20)

# Set label size - this modifies both x and y axis labels
ax.tick_params(labelsize=20)

# To modify x and y labels size use
ax.set_xlabel('Time', fontsize=20)
ax.set_ylabel('Nuc Volume', fontsize=20)

plt.savefig('figures/-Doxy +Cisp 48hr Nucleus volume.png', dpi=300, bbox_inches='tight')

In [ ]:
plus_doxy_minus_liv_plus_cisp_0hr = filtered_df[(filtered_df['Doxy'] == '+doxy') & (filtered_df['LIV'] == '-LIV') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['Time'] == '0hr')]

In [ ]:
plus_doxy_minus_liv_plus_cisp_0hr

In [ ]:
plus_doxy_minus_liv_plus_cisp_0hr['AF488-T3 av_signal_in_nuc_area_3D_removed_background'].mean()

In [ ]:
plus_doxy_minus_liv_plus_cisp_48hr = filtered_df[(filtered_df['Doxy'] == '+doxy') & (filtered_df['LIV'] == '-LIV') & (filtered_df['Treatment'] == '+Cisp') & (filtered_df['Time'] == '48hr')]
plus_doxy_minus_liv_plus_cisp_48hr

In [ ]:
plus_doxy_minus_liv_plus_cisp_48hr['AF488-T3 av_signal_in_nuc_area_3D_removed_background'].mean()


In [ ]:
filtered_df

## Nucleus Volume Comparison

In [ ]:
filtered_df_0h = filtered_df[(filtered_df['Time'] == '0hr') & (filtered_df['Treatment'] == '+Cisp')]
# Filtering the data based on the conditions
filtered_df_0h = filtered_df_0h[((filtered_df_0h['Doxy'] == '+doxy') & (filtered_df_0h['AF594-T2 has ring'] == True)) |
                                ((filtered_df_0h['Doxy'] == '-doxy') & (filtered_df_0h['AF594-T2 has ring'] == False))]

filtered_df_48h = filtered_df[(filtered_df['Time'] == '48hr') & (filtered_df['Treatment'] == '+Cisp')]
# Filtering the data based on the conditions
filtered_df_48h = filtered_df_48h[((filtered_df_48h['Doxy'] == '+doxy') & (filtered_df_48h['AF594-T2 has ring'] == True)) |
                                ((filtered_df_48h['Doxy'] == '-doxy') & (filtered_df_48h['AF594-T2 has ring'] == False))]
filtered_df_48h

In [ ]:
# Custom colors
blue = (0.298, 0.447, 0.690)  # a desaturated blue color
orange = (1.0, 0.498, 0.055)  # a desaturated orange color
pal = [blue, orange]

sns.set(font_scale=2, rc={'axes.facecolor':(0,0,0,0), 'figure.facecolor':(0,0,0,0)}, style="ticks")

significanceComparisons = [(('+doxy', '-LIV'), ('+doxy', '+LIV')),
                           (('-doxy', '-LIV'), ('-doxy','+LIV')),
                           (('-doxy', '-LIV'), ('+doxy','-LIV')),
                           (('-doxy', '+LIV'), ('+doxy','+LIV'))]

fig_args = {'x': 'Doxy',
            'y': 'Nucleus_volume, cubic_micrometre',
            'hue':'LIV',
            'data': filtered_df_0h,
            'dodge': True,
            'palette': pal}

configuration = {'test':'Mann-Whitney',
                 'comparisons_correction':None,
                 'text_format':'star'}

fig, ax = plt.subplots(1,1, figsize=(15,10))

# Barplot
sns.barplot(ax=ax, **fig_args)

# Calculating and adding averages above the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                xytext = (0, 10),
                textcoords = 'offset points')

# Annotating significance
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# Title
ax.set_title('Volume comparison \n0 hr \nwith ring and no ring correction')

# Move legend and remove its background
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
legend = ax.legend_
legend.get_frame().set_alpha(0)

# Save figure
plt.savefig('figures/0h_volume_comparison_with_ring-no-ring_correction.png')

plt.show()  # Display the plot


In [ ]:
fig_args = {'x': 'Doxy',
            'y': 'Nucleus_volume, cubic_micrometre',
            'hue':'LIV',
            'data': filtered_df_48h,
            'dodge': True,
            'palette': pal}

configuration = {'test':'Mann-Whitney',
                 'comparisons_correction':None,
                 'text_format':'star'}

fig, ax = plt.subplots(1,1, figsize=(15,10))

# Barplot
sns.barplot(ax=ax, **fig_args)

# Calculating and adding averages above the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                xytext = (0, 10),
                textcoords = 'offset points')

# Annotating significance
annotator = Annotator(ax=ax, pairs=significanceComparisons, **fig_args)
annotator.configure(**configuration).apply_test().annotate()

# Title
ax.set_title('Volume comparison \n48 hr \nwith ring and no ring correction')

# Move legend and remove its background
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
legend = ax.legend_
legend.get_frame().set_alpha(0)

# Save figure
plt.savefig('figures/48h_volume_comparison_with_ring-no-ring_correction.png')

plt.show()  # Display the plot